# Import Libraries

In [64]:
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score

from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier
from art.attacks.inference.attribute_inference import AttributeInferenceBlackBox

import warnings
warnings.filterwarnings("ignore")

# Load Data

In [103]:
def create_dataset(num_samples=1000):
    X = np.random.rand(num_samples, 9)
    binary_feature = np.random.randint(2, size=(num_samples, 1))
    X = np.hstack((X, binary_feature))
    Y = (X[:, 0] + X[:, 1] > 1).astype(int)
    return X, Y

In [104]:
x_train, y_train = create_dataset()
x_test, y_test = create_dataset()

# Train

In [105]:
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [106]:
predictions = dt.predict(x_test)

In [107]:
print('Model accuracy score:', accuracy_score(y_test, predictions))

Model accuracy score: 0.962


In [108]:
classifier = ScikitlearnDecisionTreeClassifier(model=dt)

# Attack

In [109]:
attack_ratio = 0.5

In [110]:
attack_train_size = int(len(x_train) * attack_ratio)

In [111]:
attack_x_train = x_train[:attack_train_size]
attack_y_train = y_train[:attack_train_size]

In [112]:
attack_x_test = x_test[attack_train_size:]
attack_y_test = y_test[attack_train_size:]

In [114]:
attack_feature_idx = 9

In [115]:
attack_x_test_predictions = np.argmax(classifier.predict(attack_x_test), axis=1).reshape(-1, 1)

In [116]:
attack_x_test_feature = attack_x_test[:, attack_feature_idx].reshape(-1, 1)

In [117]:
attack_x_test = np.delete(attack_x_test, attack_feature_idx, 1)

In [118]:
att_inf = AttributeInferenceBlackBox(estimator=classifier, attack_feature=attack_feature_idx)
att_inf.fit(attack_x_train)

In [119]:
Counter(x_train[:, attack_feature_idx])

Counter({1.0: 512, 0.0: 488})

In [120]:
feature_values = [1.0, 0.0]

# Results

In [121]:
inferred_train = att_inf.infer(x=attack_x_test, pred=attack_x_test_predictions, values=feature_values)

In [122]:
attack_train_acc = np.mean(np.isclose(inferred_train, np.round(attack_x_test_feature, decimals=1)))
print(attack_train_acc)

0.499856


In [143]:
print("Gerçek gizli özellikler (binary): ", attack_x_test_predictions.reshape(1, -1)[0][:5].tolist())
print("Tahmin edilen gizli özellikler (binary): ", inferred_train[:5].tolist())

Gerçek gizli özellikler (binary):  [1, 0, 0, 0, 1]
Tahmin edilen gizli özellikler (binary):  [1.0, 0.0, 0.0, 0.0, 0.0]
